In [1]:
import ECGDataset 
import Models 
import Net
from train_test_validat import *
from self_attention import *
import matplotlib.pyplot as plt
import ecg_plot

import torch
import torch.utils.data as Data
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import random
import pandas as pd

import time
import os
import gc

random_seed = 2
torch.manual_seed(random_seed)    # reproducible
torch.cuda.manual_seed_all(random_seed)
random.seed(random_seed)
np.random.seed(random_seed)


In [2]:
EcgChannles_num = 12
EcgLength_num = 5000
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda:0


In [3]:
ALLDataset = ECGDataset.ECG_Dataset_Init('/workspace/data/Preprocess_HTN/data_like_pxl/',filter_age=45)
ALLDataset.report()  # type: ignore
torch.cuda.empty_cache()# 清空显卡cuda
NET = [Net.MLBFNet(True,res = True,se = True,Dropout_rate = 0.25) ] # type: ignore
test_dataset = ECGDataset.ECG_Dataset(ALLDataset.testECGs,ALLDataset.testLabels,ALLDataset.testDf,preprocess= True,onehot_lable= False)  # type: ignore


              HTN        NHTN   
   nums       1871      103183  


            orginal            QC         
   nums      105054          65944        
              HTN        NHTN   
   nums       1344      64600   


            orginal   filtered department 
   nums      65944           13953        
              HTN        NHTN   
   nums       1344      12609   


            orginal     removed ages NaN  
   nums      13953           13953        
              HTN        NHTN   
   nums       1344      12609   


            orginal      filtered ages    
   nums      13953            5269        
              HTN        NHTN   
   nums       1344       3925   


            orginal   removed diagnose NaN
   nums       5269            5266        
              HTN        NHTN   
   nums       1344       3922   


              HTN        NHTN   
   ALL        1344       3922   
 testset      346        1606   
 T&V set      998        2316   
              HTN        NHTN 

In [4]:
test_dataset.infos.to_csv('./testdataset.csv')

In [5]:
testmodel = NET[0].to(DEVICE)

In [6]:
Models_path = "./model/20221206_095840/"
for i in range(5):
    testmodel.load_state_dict(torch.load(Models_path+"/parameter_EarlyStoping_"+str(i)+".pt"))
    test_dataloader = Data.DataLoader(dataset=test_dataset, shuffle= False,batch_size=128)
    test_acc = []   
    criterion = torch.nn.CrossEntropyLoss() 
    y_true,y_pred,test_loss,test_acc = eval_model(test_dataloader,criterion,testmodel,DEVICE) # 测试模型
    print('loss =',test_loss,'acc =',test_acc)
    print('f1_macro =',f1_score(y_true, y_pred, average='macro')) 
    print('f1_micro =',f1_score(y_true, y_pred, average='micro')) 
    cm = confusion_matrix(y_true=y_true, y_pred=y_pred)
    print("Confusion Matrix: ")
    print(cm)
    df_grade = pd.DataFrame(test_dataset.infos['ecgFN'].tolist(),columns=['file'])
    df_grade = pd.concat([df_grade, pd.DataFrame(y_true,columns=['true'])],axis=1)
    df_grade = pd.concat([df_grade, pd.DataFrame(y_pred,columns=['pred'])],axis=1)
    df_grade.to_csv(Models_path+"/answer"+str(i)+".csv")

loss = 0.4064604863524437 acc = 0.90380859375
f1_macro = 0.8380812956365976
f1_micro = 0.9052254098360656
Confusion Matrix: 
[[1512   94]
 [  91  255]]
loss = 0.4221753478050232 acc = 0.88818359375
f1_macro = 0.8061115614696712
f1_micro = 0.8888319672131147
Confusion Matrix: 
[[1505  101]
 [ 116  230]]
loss = 0.4038619417697191 acc = 0.90283203125
f1_macro = 0.8395936557145626
f1_micro = 0.9026639344262294
Confusion Matrix: 
[[1493  113]
 [  77  269]]
loss = 0.40635606087744236 acc = 0.9013671875
f1_macro = 0.83821887213847
f1_micro = 0.9026639344262294
Confusion Matrix: 
[[1497  109]
 [  81  265]]
loss = 0.4345214832574129 acc = 0.875
f1_macro = 0.800669612963707
f1_micro = 0.875
Confusion Matrix: 
[[1450  156]
 [  88  258]]


In [7]:
answer1 = pd.read_csv(Models_path+"/answer0.csv")
answer1.rename(columns={'pred':'pred1'},inplace=True) 
answer2 = pd.read_csv(Models_path+"/answer1.csv")
answer2.rename(columns={'pred':'pred2'},inplace=True) 
answer3 = pd.read_csv(Models_path+"/answer2.csv")
answer3.rename(columns={'pred':'pred3'},inplace=True) 
answer4 = pd.read_csv(Models_path+"/answer3.csv")
answer4.rename(columns={'pred':'pred4'},inplace=True) 
answer5 = pd.read_csv(Models_path+"/answer4.csv")
answer5.rename(columns={'pred':'pred5'},inplace=True) 

answer1 = pd.merge(answer1,answer2.loc[:,['file','pred2']],how='outer',on='file')
answer1 = pd.merge(answer1,answer3.loc[:,['file','pred3']],how='outer',on='file')
answer1 = pd.merge(answer1,answer4.loc[:,['file','pred4']],how='outer',on='file')
answer1 = pd.merge(answer1,answer5.loc[:,['file','pred5']],how='outer',on='file')

In [8]:
answer1

,Unnamed: 0,file,true,pred1,pred2,pred3,pred4,pred5
0,0,21-1-103_34-1_NHTN.npy,0,0,0,0,0,0
1,1,21-1-105_79-0_HTN.npy,1,1,1,1,1,1
2,2,21-1-106_26-0_NHTN.npy,0,0,0,0,0,0
3,3,21-1-108_34-1_NHTN.npy,0,0,0,0,0,0
4,4,21-1-1100_31-0_NHTN.npy,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
1947,1947,21-9-459_40-1_NHTN.npy,0,1,1,1,0,0
1948,1948,21-9-461_31-0_NHTN.npy,0,0,0,0,0,0
1949,1949,21-9-463_30-0_NHTN.npy,0,0,0,0,0,0
1950,1950,21-9-65_44-1_NHTN.npy,0,0,0,0,0,0


In [9]:
answer1.insert(loc=len(answer1.columns), column='ERR_Count', value=(abs(5*answer1['true'] -(answer1['pred1']+answer1['pred2']+answer1['pred3']+answer1['pred4']+answer1['pred5']))))

In [10]:
(answer1['ERR_Count']).value_counts()

0    1469
1     231
2      99
3      64
5      49
4      40
Name: ERR_Count, dtype: int64

In [11]:
HTN_errs = answer1[(answer1['true'] == 1)]
HTN_errs['ERR_Count'].value_counts()

0    170
1     57
2     42
5     31
3     27
4     19
Name: ERR_Count, dtype: int64

In [12]:
NHTN_errs = answer1[(answer1['true'] == 0)]
NHTN_errs['ERR_Count'].value_counts()

0    1299
1     174
2      57
3      37
4      21
5      18
Name: ERR_Count, dtype: int64

In [13]:
HTN_errs = answer1[(answer1['ERR_Count']>2)&(answer1['true'] == 1)]

In [14]:
from tqdm import tqdm
csv_data = pd.DataFrame()
count = 0 
for index in tqdm((HTN_errs['Unnamed: 0']).tolist()):
    inf = test_dataset.infos.iloc[index]
    ECG,label = test_dataset.__getitem__(index)
    file_name = inf['name']+'_'+inf['gender']+'_'+str(inf['ages'])+'_'+inf['department'] +'_'+str(inf['diagnose'])+'_'+inf['ID']+'_'+inf['date']
    ecg_plot.plot(ECG*3500/1000, sample_rate = 500, title = file_name,row_height= 10,show_grid=True,show_separate_line=True)
    file_name = str(count)+'_'+str(HTN_errs.loc[index,'file'])+'_'+file_name+'_'+str(HTN_errs.loc[index,'ERR_Count'])
    ecg_plot.save_as_png(file_name,Models_path+"/ERR/",dpi = 100)
    csv_data = csv_data.append(inf)
    count = count + 1

100%|██████████| 77/77 [04:32<00:00,  3.54s/it]


In [15]:
csv_data = csv_data.reset_index(drop=True)

In [16]:
csv_data

,num,name,ages,gender,department,diagnose,ID,date,ecgFN,q_sum
0,1191.0,李丽琴,45.0,女,内分泌糖尿病科病房,1.0,848037,2021-01-06 16:57:06,21-1-1191_45-0_HTN.npy,0.0
1,1311.0,李杰,28.0,男,心血管内科病房,1.0,849760,2021-01-19 09:40:16,21-1-1311_28-1_HTN.npy,0.0
2,1472.0,杨国富,54.0,男,心血管内科病房,1.0,851140,2021-01-26 19:29:04,21-1-1472_54-1_HTN.npy,0.0
3,1489.0,杨希,35.0,女,心血管内科病房,1.0,846397,2021-01-06 15:57:22,21-1-1489_35-0_HTN.npy,0.0
4,209.0,刘汤豪,22.0,男,内分泌糖尿病科病房,1.0,849816,2021-01-19 11:15:35,21-1-209_22-1_HTN.npy,0.0
...,...,...,...,...,...,...,...,...,...,...
72,2652.0,苏宝崇,67.0,男,心血管内科病房,1.0,883055,2021-08-12 09:26:03,21-8-2652_67-1_HTN.npy,0.0
73,2827.0,蔡聪敏,72.0,男,心血管内科病房,1.0,729993,2021-08-20 18:26:11,21-8-2827_72-1_HTN.npy,0.0
74,4190.0,黄莉玲,37.0,女,内分泌糖尿病科病房,1.0,883488,2021-08-15 15:57:56,21-8-4190_37-0_HTN.npy,0.0
75,89.0,余繁斌,69.0,男,心血管内科病房,1.0,518243,2021-08-04 11:09:10,21-8-89_69-1_HTN.npy,0.0


In [17]:
HTN_errs

,Unnamed: 0,file,true,pred1,pred2,pred3,pred4,pred5,ERR_Count
7,7,21-1-1191_45-0_HTN.npy,1,0,0,0,1,0,4
14,14,21-1-1311_28-1_HTN.npy,1,0,0,0,0,0,5
24,24,21-1-1472_54-1_HTN.npy,1,0,0,0,0,0,5
25,25,21-1-1489_35-0_HTN.npy,1,0,0,0,0,0,5
52,52,21-1-209_22-1_HTN.npy,1,0,0,1,0,0,4
...,...,...,...,...,...,...,...,...,...
1754,1754,21-8-2652_67-1_HTN.npy,1,0,0,0,0,1,4
1772,1772,21-8-2827_72-1_HTN.npy,1,0,0,0,1,0,4
1868,1868,21-8-4190_37-0_HTN.npy,1,0,0,0,1,0,4
1912,1912,21-8-89_69-1_HTN.npy,1,0,0,1,0,0,4


In [18]:
csv_data = pd.merge(csv_data,(HTN_errs.loc[:,['file','ERR_Count']].reset_index(drop=True).rename(columns={'file':'ecgFN'})),how='inner',on='ecgFN')
csv_data.to_csv(Models_path+'/高血压分错.csv')

In [19]:
csv_data

,num,name,ages,gender,department,diagnose,ID,date,ecgFN,q_sum,ERR_Count
0,1191.0,李丽琴,45.0,女,内分泌糖尿病科病房,1.0,848037,2021-01-06 16:57:06,21-1-1191_45-0_HTN.npy,0.0,4
1,1311.0,李杰,28.0,男,心血管内科病房,1.0,849760,2021-01-19 09:40:16,21-1-1311_28-1_HTN.npy,0.0,5
2,1472.0,杨国富,54.0,男,心血管内科病房,1.0,851140,2021-01-26 19:29:04,21-1-1472_54-1_HTN.npy,0.0,5
3,1489.0,杨希,35.0,女,心血管内科病房,1.0,846397,2021-01-06 15:57:22,21-1-1489_35-0_HTN.npy,0.0,5
4,209.0,刘汤豪,22.0,男,内分泌糖尿病科病房,1.0,849816,2021-01-19 11:15:35,21-1-209_22-1_HTN.npy,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...
72,2652.0,苏宝崇,67.0,男,心血管内科病房,1.0,883055,2021-08-12 09:26:03,21-8-2652_67-1_HTN.npy,0.0,4
73,2827.0,蔡聪敏,72.0,男,心血管内科病房,1.0,729993,2021-08-20 18:26:11,21-8-2827_72-1_HTN.npy,0.0,4
74,4190.0,黄莉玲,37.0,女,内分泌糖尿病科病房,1.0,883488,2021-08-15 15:57:56,21-8-4190_37-0_HTN.npy,0.0,4
75,89.0,余繁斌,69.0,男,心血管内科病房,1.0,518243,2021-08-04 11:09:10,21-8-89_69-1_HTN.npy,0.0,4


In [10]:
pred = torch.tensor([0.2, 0.6, 0.8])
y = torch.tensor([0, 1, 1])
pred.ge(0.5).int()

tensor([0, 1, 1], dtype=torch.int32)

In [13]:
torch.tensor(0.5)

tensor(0.5000)

In [16]:
pred = torch.tensor([[0, 0, 1], [0, 1, 0]])
taget = torch.tensor([[0, 0, 1], [0, 1, 0]])

In [17]:
num_correct = (pred == taget).sum().item()

In [25]:
accuracy = (sum(row.all().int().item() for row in (pred == taget)))/(len(taget)*1.0)

In [24]:
accuracy

1.0

In [3]:

data1=torch.randn((1,3,2,4))
print('data1值为：',data1)
 
data2=data1.view(1,2*3,4)
print('data2值为：',data2)

data3=data1.view(1,3*2,4)
print('data3值为：',data3)
data3=data3.permute(0,2 ,1)
print('data3值为：',data3)
data3=data3.permute(0,2 ,1)
print('data3值为：',data3)
data3=data3.view(1,2,3,4)
print('data3值为：',data3)


data1值为： tensor([[[[-0.8663, -0.2656,  0.1665,  0.7911],
          [ 0.8560,  0.4094, -1.1371, -1.2034]],

         [[ 1.1414, -0.6354, -1.4702, -0.2134],
          [-0.8707,  1.6159, -0.2356,  0.9444]],

         [[ 2.1134, -0.9754,  0.1757, -0.1319],
          [-0.2735,  0.3355,  0.1885,  2.1432]]]])
data2值为： tensor([[[-0.8663, -0.2656,  0.1665,  0.7911],
         [ 0.8560,  0.4094, -1.1371, -1.2034],
         [ 1.1414, -0.6354, -1.4702, -0.2134],
         [-0.8707,  1.6159, -0.2356,  0.9444],
         [ 2.1134, -0.9754,  0.1757, -0.1319],
         [-0.2735,  0.3355,  0.1885,  2.1432]]])
data3值为： tensor([[[-0.8663, -0.2656,  0.1665,  0.7911],
         [ 0.8560,  0.4094, -1.1371, -1.2034],
         [ 1.1414, -0.6354, -1.4702, -0.2134],
         [-0.8707,  1.6159, -0.2356,  0.9444],
         [ 2.1134, -0.9754,  0.1757, -0.1319],
         [-0.2735,  0.3355,  0.1885,  2.1432]]])
data3值为： tensor([[[-0.8663,  0.8560,  1.1414, -0.8707,  2.1134, -0.2735],
         [-0.2656,  0.4094, -0.6354,